In [1]:
import os
from glob import glob
from datetime import datetime

import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

from tensorflow.keras import datasets

import matplotlib.pyplot as plt
%matplotlib inline
%load_ext tensorboard

In [2]:
num_epochs = 20
batch_size = 32

learning_rate = 0.001
dropout_rate = 0.5

input_shape = (32, 32, 3)
num_classes = 10

In [3]:
#model
inputs = layers.Input(input_shape)
net = layers.Conv2D(32, (3, 3), padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(dropout_rate)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs=inputs, outputs=net, name='Basic_CNN')

In [4]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate), # optimization
             loss='sparse_categorical_crossentropy', # loss function
             metrics=['accuracy']) # metrics / accuracy

In [5]:
# Data Preprocess

train_paths = sorted(glob('/Users/sengjeawang/Desktop/data/dataset/cifar/train/*.png'))[:100]
test_paths = sorted(glob('/Users/sengjeawang/Desktop/data/dataset/cifar/test/*.png'))[:100]

len(train_paths), len(test_paths)

(100, 100)

In [6]:
def get_class_name(path):
    return path.split('_')[-1].replace('.png', '')

train_labels = [get_class_name(path) for path in train_paths]
class_names = np.unique(train_labels)

In [7]:
def get_label(path):
    fname = tf.strings.split(path, '_')[-1]
    lbl_name = tf.strings.regex_replace(fname, '.png', '')
    onehot = tf.cast(lbl_name == class_names, tf.uint8)
    return tf.argmax(onehot)

In [8]:
def load_image_label(path):
    gfile = tf.io.read_file(path)
    image = tf.io.decode_image(gfile)
    
    image = tf.cast(image, tf.float32) / 255.
    
    label = get_label(path)
    return image, label

In [9]:
def image_preprocess(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    return image, label

In [10]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [11]:
train_dataset = tf.data.Dataset.from_tensor_slices(train_paths)
train_dataset = train_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.map(image_preprocess, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.shuffle(buffer_size=len(train_paths))
train_dataset = train_dataset.repeat()

In [12]:
test_dataset = tf.data.Dataset.from_tensor_slices(test_paths)
test_dataset = test_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
test_dataset = test_dataset.batch(batch_size)
test_dataset = test_dataset.repeat()

In [13]:
# Learning Rate Scheduler
import math

def scheduler(epoch):
    if epoch < 10:
        return 0.001
    else:
        return 0.001 * math.exp(0.1 * (10 - epoch))

In [14]:
for epoch in range(10, 20):
    print(0.001 * math.exp(0.1 * (10 - epoch)))

0.001
0.0009048374180359595
0.0008187307530779819
0.0007408182206817179
0.0006703200460356394
0.0006065306597126335
0.0005488116360940264
0.0004965853037914095
0.0004493289641172216
0.00040656965974059914


In [15]:
learning_rate_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler, verbose=1)

In [16]:
# Training

steps_per_epoch = len(train_paths) // batch_size
validation_steps = len(test_paths) // batch_size

model.fit_generator(
    train_dataset,
    steps_per_epoch=steps_per_epoch,
    validation_data=test_dataset,
    validation_steps=validation_steps,
    epochs=num_epochs,
    callbacks=[learning_rate_scheduler]
)

/Users/sengjeawang/opt/anaconda3/lib/python3.8/site-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20

Epoch 00001: LearningRateScheduler setting learning rate to 0.001.
3/3 [==============================] - 1s 160ms/step - loss: 2.3379 - accuracy: 0.1250 - val_loss: 2.2866 - val_accuracy: 0.1042
Epoch 2/20

Epoch 00002: LearningRateScheduler setting learning rate to 0.001.
3/3 [==============================] - 0s 83ms/step - loss: 2.3370 - accuracy: 0.1250 - val_loss: 2.3170 - val_accuracy: 0.0729
Epoch 3/20

Epoch 00003: LearningRateScheduler setting learning rate to 0.001.
3/3 [==============================] - 0s 100ms/step - loss: 2.3739 - accuracy: 0.0625 - val_loss: 2.3155 - val_accuracy: 0.0729
Epoch 4/20

Epoch 00004: LearningRateScheduler setting learning rate to 0.001.
3/3 [==============================] - 0s 78ms/step - loss: 2.2809 - accuracy: 0.0735 - val_loss: 2.3183 - val_accuracy: 0.0729
Epoch 5/20

Epoch 00005: LearningRateScheduler setting learning rate to 0.001.
3/3 [==============================] - 0s 102ms/step - loss: 2.3203 - accuracy: 0.1146 - va